<a href="https://colab.research.google.com/github/arutraj/ML_Basics/blob/main/Model_Checkpointing_in_Convolutional_Neural_Network_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps to build CNN to solve Emergency vs Non-Emergency vehicle classification problem

1. Loading the Dataset
2. Pre-processing the Data
3. Creating Training and Validation set
4. Defining the Model Architecture
5. Compiling the Model
6. Training the Model
<ol>Model Checkpointing</ol>
7. Evaluating model performance

## 1. Loading the Dataset

In [3]:
# import necessary libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# importing layers from keras
from keras.layers import Dense, InputLayer, BatchNormalization, Dropout
from keras.models import Sequential
# importing adam optimizer from keras optimizer module
from keras.optimizers import Adam

# train_test_split to create training and validation set
from sklearn.model_selection import train_test_split
# accuracy_score to calculate the accuracy of predictions
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# unzipping the Dataset.zip file
#!unzip /content/drive/MyDrive/Dataset.zip
!ls /content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/*jpg |wc -l

2352


In [5]:
# create random number generator
seed = 42

In [6]:
# reading the csv file
data = pd.read_csv('/content/emergency_classification.csv')

In [7]:
# load images and store it in numpy array

# empty list to store the images
X = []
# iterating over each image
for img_name in data.image_names:
    # loading the image using its name
    img = plt.imread('/content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/' + img_name)
    # saving each image in the list
    X.append(img)

# converting the list of images into array
X=np.array(X)

# storing the target variable in separate variable
y = data.emergency_or_not.values

In [8]:
# shape of the images
X.shape

(2352, 224, 224, 3)

## 2. Pre-processing the Data

In [9]:
# normalizing the pixel values
X = X / X.max()

## 3. Creating Training and Validation set

In [10]:
# creating a training and validation set
X_train, X_valid, y_train, y_valid=train_test_split(X,y,test_size=0.3, random_state=seed)

In [11]:
# shape of training and validation set
(X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape)

(((1646, 224, 224, 3), (1646,)), ((706, 224, 224, 3), (706,)))

## 4. Defining the Model Architecture

In [12]:
# importing the convolutional and flatten layer from keras
from keras.layers import Conv2D, Flatten, MaxPool2D

In [13]:
# defining the model architecture with maxpool layer

model = Sequential()

model.add(InputLayer(input_shape=(224, 224, 3)))

model.add(Conv2D(50, (5, 5), activation='relu', strides=(1, 1), padding='valid'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(Conv2D(50, (5, 5), activation='relu', strides=(1, 1), padding='valid'))
model.add(MaxPool2D(pool_size=(4, 4), padding='valid'))

model.add(Flatten())

model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.6))
model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.6))
model.add(Dense(units=1, activation='sigmoid'))

In [14]:
# printing summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 50)      3800      
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 50)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 50)      62550     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 26, 26, 50)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 33800)             0         
                                                                 
 dense (Dense)               (None, 100)               3

## 5. Compiling the Model

In [15]:
# compiling the model
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

## 6. Training the Model

In [16]:
# defining model checkpointing
from keras.callbacks import ModelCheckpoint

# defining the path to save the model
path = 'best_model.hdf5'

# defining model checkpointing
checkpoint = ModelCheckpoint(path, monitor='val_accuracy', verbose = 1, save_best_only = True, mode='max')

# defining callback list
callback_list = [checkpoint]

In [17]:
# training the model for 100 epochs
model_history = model.fit(X_train, y_train, epochs=100, batch_size=128,validation_data=(X_valid,y_valid), callbacks=callback_list)

Epoch 1/100
13/13 [==============================] - ETA: 0s - loss: 0.9775 - accuracy: 0.5723
Epoch 1: val_accuracy improved from -inf to 0.69263, saving model to best_model.hdf5
13/13 [==============================] - 25s 959ms/step - loss: 0.9775 - accuracy: 0.5723 - val_loss: 0.6720 - val_accuracy: 0.6926
Epoch 2/100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - ETA: 0s - loss: 0.7554 - accuracy: 0.6634
Epoch 2: val_accuracy did not improve from 0.69263
13/13 [==============================] - 3s 232ms/step - loss: 0.7554 - accuracy: 0.6634 - val_loss: 1.3989 - val_accuracy: 0.4844
Epoch 3/100
13/13 [==============================] - ETA: 0s - loss: 0.7334 - accuracy: 0.6713
Epoch 3: val_accuracy did not improve from 0.69263
13/13 [==============================] - 3s 233ms/step - loss: 0.7334 - accuracy: 0.6713 - val_loss: 0.6601 - val_accuracy: 0.6771
Epoch 4/100
13/13 [==============================] - ETA: 0s - loss: 0.6481 - accuracy: 0.6926
Epoch 4: val_accuracy improved from 0.69263 to 0.74363, saving model to best_model.hdf5
13/13 [==============================] - 3s 241ms/step - loss: 0.6481 - accuracy: 0.6926 - val_loss: 0.5397 - val_accuracy: 0.7436
Epoch 5/100
13/13 [==============================] - ETA: 0s - loss: 0.5909 - accuracy: 0.7467
Epoch 5: val_accuracy did not improve from 0.7436

## 7. Evaluating model performance

In [18]:
model.load_weights('best_model.hdf5')

In [19]:
# accuracy on training set
prediction = model.predict(X_train)
prediction = prediction.reshape(-1,)

# converting probabilities to classes
prediction_int = prediction >= 0.5
prediction_int = prediction_int.astype(int)
print('Accuracy on training set:', accuracy_score(y_train, prediction_int))

52/52 [==============================] - 2s 22ms/step
Accuracy on training set: 0.9975698663426489


In [20]:
# accuracy on validation set
prediction = model.predict(X_valid)
prediction = prediction.reshape(-1,)

# converting probabilities to classes
prediction_int = prediction >= 0.5
prediction_int = prediction_int.astype(int)
print('Accuracy on validation set:', accuracy_score(y_valid, prediction_int))

23/23 [==============================] - 1s 23ms/step
Accuracy on validation set: 0.8626062322946175
